This file will be used to create a capstone project for IBM Data Science course.

In [1]:
import pandas as pd
import numpy as np
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [2]:
#df = pd.read_csv('https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv')
#df.head()

In [3]:
#df.describe()

In [4]:
#df.info()

In [5]:
#df['ROADCOND'].value_counts()

In [6]:
#df.columns